## Hyperparameters tests

This notebook was written in order to find the combination of hidden layers $l$ and neurons $w$ that leads to the lowest error between the truth trajectories and the ones predicted by our PINN. Run every cell of this notebook in order to obtain the wanted results. The data used to for this notebook are on the same folder and can be found in the file 'k=50_L=7_N=5_M=1.csv'

In [1]:
import matplotlib.pyplot as plt
import torch
from scipy.integrate import solve_ivp
from torch import nn
import numpy as np
%matplotlib inline

In [2]:
X_SCALE = 3.0
V_SCALE = 50.0 * X_SCALE**2
F_SCALE = 30.0 # V_SCALE / X_SCALE
GRAD_F_SCALE = F_SCALE / X_SCALE

define ANN and losses

In [3]:
from torch.autograd import grad
from functorch import vmap, vjp
from functorch import jacrev, jacfwd

class NNApproximator(nn.Module):
  def __init__(self, dim_input = 6, dim_output = 2, num_hidden = 2, dim_hidden = 1, activation=nn.Tanh()):
    super().__init__()

    self.dim_input = dim_input

    self.layer_in = nn.Linear(4, dim_hidden)
    # self.layer_in = nn.Linear(dim_input, dim_hidden)
    self.layer_out = nn.Linear(dim_hidden, dim_output)
    self.k = nn.Parameter(torch.tensor(50.0, requires_grad=False))

    num_middle = num_hidden - 1
    self.middle_layers = nn.ModuleList(
        [nn.Linear(dim_hidden, dim_hidden) for _ in range(num_middle)]
    )
    self.activation = activation

  def forward(self, x):
    x_unit = x.reshape((-1,self.dim_input)) / X_SCALE
    s = torch.hstack((x_unit[:, 0:2] - x_unit[:, 2:4], x_unit[:, 4:6] - x_unit[:, 2:4]))
    out = self.activation(self.layer_in(s))
    for layer in self.middle_layers:
      out = self.activation(layer(out))
    return self.layer_out(out) * F_SCALE

  def _get_force_truth(self, x):
    k = 50.0
    g = torch.tensor([[0.0, -9.81]])
    L0 = 5.0
    x1 = x[:,:2]
    x2 = x[:,2:4]
    x3 = x[:,4:]
    dx1 = x2 - x1
    dx2 = x3 - x2
    dx1_norm = torch.sqrt(torch.sum(dx1 ** 2, dim=1))[:,None]
    dx2_norm = torch.sqrt(torch.sum(dx2 ** 2, dim=1))[:,None]
    f1 = -k * (dx1_norm - L0) * (dx1 / dx1_norm)
    f2 = k * (dx2_norm - L0) * (dx2 / dx2_norm)
    F = f2
    F += g
    F += f1
    return F
  def jacobian(self, x):
    jac = vmap(jacrev(self.forward))
    return jac(x).squeeze()

In [4]:
def compute_data_loss_force(model, x_tr, y_tr):
  return 0.5 * torch.mean((model.forward(x_tr).flatten() - y_tr.flatten()) ** 2) / (F_SCALE ** 2)

In [5]:
def compute_PINN_loss(model, x, k):
    F_dot = model.jacobian(x)
    s1 = x[:, 0:2] - x[:, 2:4]
    s2 = x[:, 4:6] - x[:, 2:4]

    s1 = s1 / torch.norm(s1, dim=1)[:, None]
    s2 = s2 / torch.norm(s2, dim=1)[:, None]

    s1rot = s1 @ torch.from_numpy(np.array([[0, -1], [1, 0]]).T).float()
    s2rot = s2 @ torch.from_numpy(np.array([[0, -1], [1, 0]]).T).float()

    f1_ax = torch.einsum('ij, ijk, ik->i', s1, F_dot[:, :, 0:2], s1) - k
    f2_ax = torch.einsum('ij, ijk, ik->i', s2, F_dot[:, :, 4:6], s2) - k
    f1_perp = torch.einsum('ij, ijk, ik->i', s1, F_dot[:, :, 0:2], s1rot)
    f2_perp = torch.einsum('ij, ijk, ik->i', s2, F_dot[:, :, 4:6], s2rot)

    loss = (f1_ax ** 2).mean() + (f2_ax ** 2).mean()
    loss += (f1_perp ** 2).mean() + (f2_perp ** 2).mean()

    return loss

Load training data

In [6]:
import pandas as pd

df = pd.read_csv('k=50_L=7_N=5_M=1.csv')
positions = df[['r0_x', 'r0_y', 'r1_x', 'r1_y', 'r2_x', 'r2_y', 'r3_x', 'r3_y','r4_x','r4_y']]
forces = df[['rddot1_x','rddot1_y','rddot2_x','rddot2_y','rddot3_x','rddot3_y']]

position_stack = np.vstack((
    positions[['r0_x','r0_y','r1_x','r1_y','r2_x','r2_y']].to_numpy(),
    positions[['r1_x','r1_y','r2_x','r2_y','r3_x','r3_y']].to_numpy(),
    positions[['r2_x','r2_y','r3_x','r3_y','r4_x','r4_y']].to_numpy()
))
forces_stack = np.vstack((
    forces[['rddot1_x','rddot1_y']].to_numpy(),
    forces[['rddot2_x','rddot2_y']].to_numpy(),
    forces[['rddot3_x','rddot3_y']].to_numpy()
))

x = torch.from_numpy(position_stack.copy()).float()
F = torch.from_numpy(forces_stack.copy()).float()

In [7]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

def train_model(model, data_loss_fn, PINN_loss_fn, learning_rate=0.0001, max_epochs=1000, USE_BFGS=False, weight_decay=5e-5):
  model.train()
  tr_losses = []
  data_losses = []
  PINN_losses = []

  # reference on torch.LBFGS usage
  # https://gist.github.com/tuelwer/0b52817e9b6251d940fd8e2921ec5e20
  # USE_BFGS = False

  if USE_BFGS:
    optimizer = torch.optim.LBFGS(model.parameters())
    print("Using BFGS optimizer ... ")
    log_iter = 10
    def closure():
        optimizer.zero_grad()
        objective = data_loss_fn(model) + PINN_loss_fn(model)
        objective.backward()
        return objective
  else:
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    print("Using Adam optimizer ... ")
    log_iter = 1000

  # scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=3000, eta_min=1e-6)
  min_loss = 1e9

  for epoch in range(max_epochs):
    data_loss = data_loss_fn(model)
    PINN_loss = PINN_loss_fn(model)
    loss = data_loss + PINN_loss
    if USE_BFGS:
      optimizer.step(closure)
    else:
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    # scheduler.step()

    if epoch % log_iter == 0:
      print(f"Epoch: {epoch} - Loss: {float(loss):>7f} - Data: {float(data_loss):>7f} - PINN: {float(PINN_loss):>7f}")
      print(f"Loss over entire dataset: {float(compute_data_loss_force(model, x, F).detach()):>7f}") # TODO: fix preprocess_x
    tr_losses.append(loss.detach().numpy())
    data_losses.append(data_loss.detach().numpy())
    PINN_losses.append(PINN_loss.detach().numpy())

    if epoch > 2000 and loss.item() < 0.99 * min_loss:
    #   torch.save(model.state_dict(), 'checkpoint.pth')
      min_loss = loss.item()

  #plt.semilogy(tr_losses, label="Total")
  #plt.semilogy(data_losses, label="Data")
  #plt.semilogy(PINN_losses, label="PINN")
  #plt.legend()

  print("Min loss: ", min_loss)

  return model, np.array(tr_losses)

Preparing training and collocation data

In [8]:
class OracleModel(NNApproximator):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    return super()._get_force_truth(x)

mdl_o = OracleModel()

In [9]:
x_tr = torch.vstack((
    x[:500,:].clone(),
    x[2000:2500,:].clone(),
    x[4000:4500,:].clone()
))
y_tr = torch.vstack((
    F[:500,:].clone(),
    F[2000:2500,:].clone(),
    F[4000:4500,:].clone()
))

print(compute_data_loss_force(mdl_o, x_tr, y_tr))

x_tr += torch.randn_like(x_tr) * (torch.std(x_tr,dim=0) * 0.003)

print(compute_data_loss_force(mdl_o, x_tr, y_tr))

# sample_qmc = get_LHS_sample(x.detach(), 1024)
samples = x.detach().numpy() #np.vstack((sample_qmc, x_tr[:, :].detach()))
samples = torch.tensor(samples)
x_collocation = samples.float().requires_grad_(True)

#print(compute_PINN_loss(mdl_o, x_collocation, 50))

tensor(0.3413)
tensor(0.3426)


Function to compute trajectories and forces

In [10]:
def getqdot(x, xdot, model, xLeft, xRight):
    N_m = x.shape[0] // 2

    x = np.concatenate((xLeft, x, xRight))
    forces = []
    for i in range(0, N_m):
        triplet = x[np.arange(2*i, 2*i + 6)]
        triplet = triplet[None,:]
        force = model.forward(torch.tensor(triplet).float())
        forces.append(force.detach().numpy())

    forces = np.concatenate(forces).reshape(-1)

    q0 = np.concatenate((x, xdot))
    qdot = np.concatenate((xdot, forces))
    return qdot

def compute_trajectory(x0, x0dot, model, xLeft, xRight):
    N_m = x0.shape[0] // 2
    q0 = np.concatenate((x0, x0dot))

    t0 = 0
    tf = 20
    Nt = 2000
    sol = solve_ivp(lambda t, q: getqdot(q[:2*N_m], q[2*N_m:], model, xLeft, xRight), [t0,tf], y0=q0, t_eval=np.linspace(t0, tf, Nt))
    y = sol.y
    t = sol.t
    return y, t

Initialize parameters for plotting 

In [11]:
# compute NN-predicted closed-loop trajectories
# modify as appropriate:
N_m = 5
xLeft  = (positions.iloc[0,:][['r0_x','r0_y']]).to_numpy()
xRight = (positions.iloc[0,:][['r4_x','r4_y']]).to_numpy()
mass_cols = ['r1_x','r1_y','r2_x','r2_y','r3_x','r3_y']

# # the vector 'x0' contains the initial positions of the *movable* masses
# # i.e. x0.shape = [N_m * 2]
x0 = (positions.iloc[0,:][mass_cols]).to_numpy()
x0dot = np.zeros_like(x0)

# to run: y, t = compute_trajectory(x0, x0dot, model, xLeft, xRight)

function to compute error between truth trajectory and the predicted one at each axis

In [12]:
def mean_err_trajectory(y_true, y_pred):
  mass_cols = ['r1_x','r1_y','r2_x','r2_y','r3_x','r3_y']
  mean_err_x = []
  mean_err_y = []
  for i in range(6):
    err = (y_true[mass_cols[i]].to_numpy()-y_pred.T[:,i])**2
    mean_err_traj_i = (1/len(err))*np.sum(err)
    if i % 2 == 0:
       mean_err_x.append(mean_err_traj_i)
    else:
      mean_err_y.append(mean_err_traj_i)
  mean_err_x = np.asarray(mean_err_x)
  mean_err_y = np.asarray(mean_err_y)
  err_traj_x = (1/len(mean_err_x))*np.sum(mean_err_x)
  err_traj_y = (1/len(mean_err_y))*np.sum(mean_err_y)
  
  return err_traj_x, err_traj_y 

function to compute the aforementioned error for a given model and number of times N

In [13]:
def compute_err_trajectory_conf(model, N):
 err_traj_x = []
 err_traj_y = []
 for i in range(N):
   model_out, tr_losses = train_model(
    model,
    data_loss_fn=lambda model: 1e3*compute_data_loss_force(model, x_tr, y_tr),
    PINN_loss_fn=lambda model: 1e0*compute_PINN_loss(model, x_collocation, 50),
    learning_rate=lr,
    max_epochs=epoch,
    weight_decay=1e-3,
   )
   y_i, t_i = compute_trajectory(x0, x0dot, model_out, xLeft, xRight)
   err_traj_x_i, err_traj_y_i  = mean_err_trajectory(positions, y_i)
   err_traj_x.append(err_traj_x_i)
   err_traj_y.append(err_traj_y_i)
   
   
  
 err_traj_x = np.asarray(err_traj_x)
 err_traj_y = np.asarray(err_traj_y)
 
 

 return np.mean(err_traj_x), np.mean(err_traj_y), np.std(err_traj_x), np.std(err_traj_y)



training NN

In [14]:
#initialize parameters 
num_layer = np.array([3, 5, 7])
hidden_dim = np.array([16, 32, 64])
epoch = 6000
lr = 9e-4
N = 5

In [15]:
#create empty list that will be fill of mean squared err in x and y axis for each parameters combination
x_err_conf = []
y_err_conf = [] 

#create empty list that will be fill of the std in x and y axis for each parameters combination
x_std_conf = []
y_std_conf = [] 

Given the possible value of hidden layers and neurons associated to the fact that we trained each model 5 times, it implies that we made 45 tests. This is really time and cost consuming especially with high number of hidden layers and neurons. To avoid those problems, we first decided to split the training in different cell marking by a subtitle refered to the configuration we use so that we are not in obligation to run everything in one time. Moreover, we bought 100 CPU in Google Colab for faster computation. 

This process was already done and the results were saved in the same folder in NPY format : 

- 'x_err_conf_multi_train.npy' : contains the MSE between the truth trajectories and the predicted ones in $\hat{x}$ axis 
- 'y_err_conf_multi_train.npy' : contains the MSE between the truth trajectories and the predicted ones in $\hat{y}$ axis 
- 'x_std_conf_multi_train.npy' : contains the std of the MSE computed in the file 'x_err_conf_multi_train.npy'
- 'y_std_conf_multi_train.npy' : contains the std of the MSE computed in the file 'y_err_conf_multi_train.npy'

hidden layers = 3 and neurons = 16

In [16]:
#creating model 
model_3_16 = NNApproximator(num_hidden=num_layer[0], dim_hidden=hidden_dim[0])

In [17]:
err_traj_x_3_16, err_traj_y_3_16, std_traj_x_3_16, std_traj_y_3_16 = compute_err_trajectory_conf(model_3_16, N)

Using Adam optimizer ... 
Epoch: 0 - Loss: 5059.267578 - Data: 56.455448 - PINN: 5002.812012
Loss over entire dataset: 0.054884
Epoch: 1000 - Loss: 5.720138 - Data: 3.635632 - PINN: 2.084506
Loss over entire dataset: 0.002489
Epoch: 2000 - Loss: 3.088752 - Data: 2.553238 - PINN: 0.535514
Loss over entire dataset: 0.001368
Epoch: 3000 - Loss: 1.689569 - Data: 1.548665 - PINN: 0.140904
Loss over entire dataset: 0.000335
Epoch: 4000 - Loss: 1.484839 - Data: 1.392528 - PINN: 0.092311
Loss over entire dataset: 0.000192
Epoch: 5000 - Loss: 1.399354 - Data: 1.330261 - PINN: 0.069092
Loss over entire dataset: 0.000143
Min loss:  1.31850004196167
Using Adam optimizer ... 
Epoch: 0 - Loss: 1.308923 - Data: 1.257309 - PINN: 0.051614
Loss over entire dataset: 0.000426
Epoch: 1000 - Loss: 1.264019 - Data: 1.222247 - PINN: 0.041773
Loss over entire dataset: 0.000062
Epoch: 2000 - Loss: 1.202668 - Data: 1.168522 - PINN: 0.034145
Loss over entire dataset: 0.000045
Epoch: 3000 - Loss: 1.180052 - Data: 

In [18]:
x_err_conf.append(err_traj_x_3_16)
y_err_conf.append(err_traj_y_3_16)
x_std_conf.append(std_traj_x_3_16)
y_std_conf.append(std_traj_y_3_16)

hidden layers = 3 and neurons = 32

In [20]:
#creating model 
model_3_32 = NNApproximator(num_hidden=num_layer[0], dim_hidden=hidden_dim[1])

In [21]:
#training the PINN N times
err_traj_x_3_32, err_traj_y_3_32, std_traj_x_3_32, std_traj_y_3_32 = compute_err_trajectory_conf(model_3_32, N)

Using Adam optimizer ... 
Epoch: 0 - Loss: 5018.904297 - Data: 13.054123 - PINN: 5005.850098
Loss over entire dataset: 0.012351
Epoch: 1000 - Loss: 4.076978 - Data: 3.484272 - PINN: 0.592706
Loss over entire dataset: 0.002336
Epoch: 2000 - Loss: 2.905474 - Data: 2.590789 - PINN: 0.314685
Loss over entire dataset: 0.001416
Epoch: 3000 - Loss: 1.641319 - Data: 1.458971 - PINN: 0.182348
Loss over entire dataset: 0.000258
Epoch: 4000 - Loss: 2.106978 - Data: 2.006055 - PINN: 0.100924
Loss over entire dataset: 0.000527
Epoch: 5000 - Loss: 1.313331 - Data: 1.249187 - PINN: 0.064145
Loss over entire dataset: 0.000077
Min loss:  1.269083857536316
Using Adam optimizer ... 
Epoch: 0 - Loss: 1.299512 - Data: 1.245766 - PINN: 0.053745
Loss over entire dataset: 0.020878
Epoch: 1000 - Loss: 1.239695 - Data: 1.191824 - PINN: 0.047872
Loss over entire dataset: 0.000038
Epoch: 2000 - Loss: 1.213892 - Data: 1.172557 - PINN: 0.041335
Loss over entire dataset: 0.000028
Epoch: 3000 - Loss: 1.190012 - Data:

In [22]:
x_err_conf.append(err_traj_x_3_32)
y_err_conf.append(err_traj_y_3_32)
x_std_conf.append(std_traj_x_3_32)
y_std_conf.append(std_traj_y_3_32)

hidden layers = 3 and neurons = 64

In [25]:
#creating model 
model_3_64 = NNApproximator(num_hidden=num_layer[0], dim_hidden=hidden_dim[2])

In [26]:
#training the PINN N times
err_traj_x_3_64, err_traj_y_3_64, std_traj_x_3_64, std_traj_y_3_64 = compute_err_trajectory_conf(model_3_64, N)

Using Adam optimizer ... 
Epoch: 0 - Loss: 5000.105469 - Data: 17.118092 - PINN: 4982.987305
Loss over entire dataset: 0.012006
Epoch: 1000 - Loss: 4.486187 - Data: 3.920775 - PINN: 0.565412
Loss over entire dataset: 0.002384
Epoch: 2000 - Loss: 1.569790 - Data: 1.385175 - PINN: 0.184615
Loss over entire dataset: 0.000200
Epoch: 3000 - Loss: 1.389145 - Data: 1.295832 - PINN: 0.093313
Loss over entire dataset: 0.000113
Epoch: 4000 - Loss: 1.315813 - Data: 1.245323 - PINN: 0.070490
Loss over entire dataset: 0.000084
Epoch: 5000 - Loss: 1.651981 - Data: 1.594893 - PINN: 0.057088
Loss over entire dataset: 0.000632
Min loss:  1.2340532541275024
Using Adam optimizer ... 
Epoch: 0 - Loss: 1.228197 - Data: 1.180070 - PINN: 0.048128
Loss over entire dataset: 0.096822
Epoch: 1000 - Loss: 1.210476 - Data: 1.167373 - PINN: 0.043102
Loss over entire dataset: 0.000027
Epoch: 2000 - Loss: 1.190297 - Data: 1.155869 - PINN: 0.034428
Loss over entire dataset: 0.000021
Epoch: 3000 - Loss: 1.177518 - Data

In [27]:
x_err_conf.append(err_traj_x_3_64)
y_err_conf.append(err_traj_y_3_64)
x_std_conf.append(std_traj_x_3_64)
y_std_conf.append(std_traj_y_3_64)

In [29]:
print(x_err_conf)
print(y_err_conf)
print(x_std_conf)
print(y_std_conf)

[0.0005777427089595671, 0.0011537579608421216, 0.0004304513503965368]
[0.008963260140918305, 0.00435669574964377, 0.011910312907729023]
[0.00017742125328560962, 0.00043612605815767605, 0.00016256329928062275]
[0.012287397742600894, 0.0014270595549179073, 0.013644463945017745]


hidden layers = 5 and neurons = 16

In [30]:
#creating model 
model_5_16 = NNApproximator(num_hidden=num_layer[1], dim_hidden=hidden_dim[0])

In [31]:
#training the PINN N times
err_traj_x_5_16, err_traj_y_5_16, std_traj_x_5_16, std_traj_y_5_16 = compute_err_trajectory_conf(model_5_16, N)

Using Adam optimizer ... 
Epoch: 0 - Loss: 5038.514648 - Data: 29.136618 - PINN: 5009.377930
Loss over entire dataset: 0.026851
Epoch: 1000 - Loss: 6.011778 - Data: 4.399876 - PINN: 1.611902
Loss over entire dataset: 0.003196
Epoch: 2000 - Loss: 4.321008 - Data: 3.726081 - PINN: 0.594927
Loss over entire dataset: 0.002495
Epoch: 3000 - Loss: 3.083440 - Data: 2.679244 - PINN: 0.404196
Loss over entire dataset: 0.001441
Epoch: 4000 - Loss: 1.815107 - Data: 1.616577 - PINN: 0.198530
Loss over entire dataset: 0.000397
Epoch: 5000 - Loss: 1.527137 - Data: 1.394936 - PINN: 0.132201
Loss over entire dataset: 0.000188
Min loss:  1.4369497299194336
Using Adam optimizer ... 
Epoch: 0 - Loss: 1.473274 - Data: 1.373726 - PINN: 0.099548
Loss over entire dataset: 0.024355
Epoch: 1000 - Loss: 1.376534 - Data: 1.292475 - PINN: 0.084059
Loss over entire dataset: 0.000107
Epoch: 2000 - Loss: 1.318334 - Data: 1.247663 - PINN: 0.070671
Loss over entire dataset: 0.000074
Epoch: 3000 - Loss: 1.310336 - Data

In [32]:
x_err_conf.append(err_traj_x_5_16)
y_err_conf.append(err_traj_y_5_16)
x_std_conf.append(std_traj_x_5_16)
y_std_conf.append(std_traj_y_5_16)

hidden layers = 5 and neurons = 32

In [34]:
#creating model 
model_5_32 = NNApproximator(num_hidden=num_layer[1], dim_hidden=hidden_dim[1])

In [35]:
#training the PINN N times
err_traj_x_5_32, err_traj_y_5_32, std_traj_x_5_32, std_traj_y_5_32 = compute_err_trajectory_conf(model_5_32, N)

Using Adam optimizer ... 
Epoch: 0 - Loss: 5012.019043 - Data: 16.959196 - PINN: 4995.060059
Loss over entire dataset: 0.015275
Epoch: 1000 - Loss: 6.593776 - Data: 5.793343 - PINN: 0.800433
Loss over entire dataset: 0.004783
Epoch: 2000 - Loss: 2.216938 - Data: 1.783142 - PINN: 0.433796
Loss over entire dataset: 0.000562
Epoch: 3000 - Loss: 1.668452 - Data: 1.452281 - PINN: 0.216171
Loss over entire dataset: 0.000246
Epoch: 4000 - Loss: 1.549940 - Data: 1.419111 - PINN: 0.130829
Loss over entire dataset: 0.000279
Epoch: 5000 - Loss: 1.451513 - Data: 1.353364 - PINN: 0.098149
Loss over entire dataset: 0.000168
Min loss:  1.3778395652770996
Using Adam optimizer ... 
Epoch: 0 - Loss: 1.371909 - Data: 1.299018 - PINN: 0.072892
Loss over entire dataset: 0.109873
Epoch: 1000 - Loss: 1.328304 - Data: 1.266394 - PINN: 0.061910
Loss over entire dataset: 0.000108
Epoch: 2000 - Loss: 1.269018 - Data: 1.218840 - PINN: 0.050178
Loss over entire dataset: 0.000071
Epoch: 3000 - Loss: 1.233514 - Data

In [36]:
x_err_conf.append(err_traj_x_5_32)
y_err_conf.append(err_traj_y_5_32)
x_std_conf.append(std_traj_x_5_32)
y_std_conf.append(std_traj_y_5_32)

hidden layers = 5 and neurons = 64

In [38]:
#creating model 
model_5_64 = NNApproximator(num_hidden=num_layer[1], dim_hidden=hidden_dim[2])

In [39]:
#training the PINN N times
err_traj_x_5_64, err_traj_y_5_64, std_traj_x_5_64, std_traj_y_5_64 = compute_err_trajectory_conf(model_5_64, N)

Using Adam optimizer ... 
Epoch: 0 - Loss: 5007.938477 - Data: 15.788406 - PINN: 4992.149902
Loss over entire dataset: 0.013005
Epoch: 1000 - Loss: 3.635131 - Data: 3.089407 - PINN: 0.545724
Loss over entire dataset: 0.001814
Epoch: 2000 - Loss: 1.683103 - Data: 1.453594 - PINN: 0.229509
Loss over entire dataset: 0.000230
Epoch: 3000 - Loss: 1.519537 - Data: 1.404221 - PINN: 0.115316
Loss over entire dataset: 0.000191
Epoch: 4000 - Loss: 1.439825 - Data: 1.358536 - PINN: 0.081289
Loss over entire dataset: 0.000158
Epoch: 5000 - Loss: 1.464949 - Data: 1.400697 - PINN: 0.064252
Loss over entire dataset: 0.000196
Min loss:  1.3315587043762207
Using Adam optimizer ... 
Epoch: 0 - Loss: 1.739513 - Data: 1.561568 - PINN: 0.177945
Loss over entire dataset: 0.289906
Epoch: 1000 - Loss: 1.304380 - Data: 1.253513 - PINN: 0.050867
Loss over entire dataset: 0.000080
Epoch: 2000 - Loss: 1.262962 - Data: 1.213628 - PINN: 0.049334
Loss over entire dataset: 0.000052
Epoch: 3000 - Loss: 1.230443 - Data

In [40]:
x_err_conf.append(err_traj_x_5_64)
y_err_conf.append(err_traj_y_5_64)
x_std_conf.append(std_traj_x_5_64)
y_std_conf.append(std_traj_y_5_64)

hidden layers = 7 and neurons = 16

In [42]:
#creating model 
model_7_16 = NNApproximator(num_hidden=num_layer[2], dim_hidden=hidden_dim[0])

In [43]:
#training the PINN N times
err_traj_x_7_16, err_traj_y_7_16, std_traj_x_7_16, std_traj_y_7_16 = compute_err_trajectory_conf(model_7_16, N)

Using Adam optimizer ... 
Epoch: 0 - Loss: 5009.409180 - Data: 11.972176 - PINN: 4997.437012
Loss over entire dataset: 0.011870
Epoch: 1000 - Loss: 5.527797 - Data: 4.203668 - PINN: 1.324129
Loss over entire dataset: 0.003030
Epoch: 2000 - Loss: 3.289555 - Data: 2.776554 - PINN: 0.513001
Loss over entire dataset: 0.001598
Epoch: 3000 - Loss: 1.748820 - Data: 1.525069 - PINN: 0.223750
Loss over entire dataset: 0.000314
Epoch: 4000 - Loss: 1.632604 - Data: 1.456432 - PINN: 0.176173
Loss over entire dataset: 0.000244
Epoch: 5000 - Loss: 1.547641 - Data: 1.414242 - PINN: 0.133400
Loss over entire dataset: 0.000204
Min loss:  1.4851361513137817
Using Adam optimizer ... 
Epoch: 0 - Loss: 1.481306 - Data: 1.377363 - PINN: 0.103943
Loss over entire dataset: 0.069908
Epoch: 1000 - Loss: 1.454004 - Data: 1.359936 - PINN: 0.094069
Loss over entire dataset: 0.000161
Epoch: 2000 - Loss: 1.409209 - Data: 1.329936 - PINN: 0.079273
Loss over entire dataset: 0.000138
Epoch: 3000 - Loss: 1.360275 - Data

In [45]:
x_err_conf.append(err_traj_x_7_16)
y_err_conf.append(err_traj_y_7_16)
x_std_conf.append(std_traj_x_7_16)
y_std_conf.append(std_traj_y_7_16)

hidden layers = 7 and neurons = 32

In [47]:
#creating model 
model_7_32 = NNApproximator(num_hidden=num_layer[2], dim_hidden=hidden_dim[1])

In [48]:
#training the PINN N times
err_traj_x_7_32, err_traj_y_7_32, std_traj_x_7_32, std_traj_y_7_32 = compute_err_trajectory_conf(model_7_32, N)

Using Adam optimizer ... 
Epoch: 0 - Loss: 4999.131348 - Data: 12.298479 - PINN: 4986.833008
Loss over entire dataset: 0.012152
Epoch: 1000 - Loss: 4.302918 - Data: 3.560537 - PINN: 0.742381
Loss over entire dataset: 0.002413
Epoch: 2000 - Loss: 2.165046 - Data: 1.848394 - PINN: 0.316652
Loss over entire dataset: 0.000640
Epoch: 3000 - Loss: 1.567306 - Data: 1.396364 - PINN: 0.170943
Loss over entire dataset: 0.000189
Epoch: 4000 - Loss: 1.471781 - Data: 1.355862 - PINN: 0.115919
Loss over entire dataset: 0.000161
Epoch: 5000 - Loss: 1.393024 - Data: 1.309388 - PINN: 0.083636
Loss over entire dataset: 0.000122
Min loss:  1.339711308479309
Using Adam optimizer ... 
Epoch: 0 - Loss: 1.332730 - Data: 1.265374 - PINN: 0.067355
Loss over entire dataset: 0.067180
Epoch: 1000 - Loss: 1.306260 - Data: 1.243746 - PINN: 0.062514
Loss over entire dataset: 0.000076
Epoch: 2000 - Loss: 1.709245 - Data: 1.541595 - PINN: 0.167651
Loss over entire dataset: 0.000145
Epoch: 3000 - Loss: 1.259292 - Data:

In [49]:
x_err_conf.append(err_traj_x_7_32)
y_err_conf.append(err_traj_y_7_32)
x_std_conf.append(std_traj_x_7_32)
y_std_conf.append(std_traj_y_7_32)

hidden layers = 7 and neurons = 64

In [51]:
#creating model 
model_7_64 = NNApproximator(num_hidden=num_layer[2], dim_hidden=hidden_dim[2])

In [52]:
#training the PINN N times
err_traj_x_7_64, err_traj_y_7_64, std_traj_x_7_64, std_traj_y_7_64 = compute_err_trajectory_conf(model_7_64, N)

Using Adam optimizer ... 
Epoch: 0 - Loss: 5011.499023 - Data: 19.891182 - PINN: 4991.607910
Loss over entire dataset: 0.015523
Epoch: 1000 - Loss: 17.542192 - Data: 16.771872 - PINN: 0.770321
Loss over entire dataset: 0.005145
Epoch: 2000 - Loss: 1.714890 - Data: 1.477640 - PINN: 0.237251
Loss over entire dataset: 0.000280
Epoch: 3000 - Loss: 1.612304 - Data: 1.440727 - PINN: 0.171576
Loss over entire dataset: 0.000242
Epoch: 4000 - Loss: 1.534532 - Data: 1.391942 - PINN: 0.142589
Loss over entire dataset: 0.000198
Epoch: 5000 - Loss: 1.428625 - Data: 1.323455 - PINN: 0.105170
Loss over entire dataset: 0.000141
Min loss:  1.3575477600097656
Using Adam optimizer ... 
Epoch: 0 - Loss: 1.724043 - Data: 1.419495 - PINN: 0.304549
Loss over entire dataset: 0.250291
Epoch: 1000 - Loss: 1.330644 - Data: 1.256859 - PINN: 0.073785
Loss over entire dataset: 0.000085
Epoch: 2000 - Loss: 1.424352 - Data: 1.248162 - PINN: 0.176190
Loss over entire dataset: 0.000051
Epoch: 3000 - Loss: 1.276904 - Da

In [53]:
x_err_conf.append(err_traj_x_7_64)
y_err_conf.append(err_traj_y_7_64)
x_std_conf.append(std_traj_x_7_64)
y_std_conf.append(std_traj_y_7_64)

Save data and export

In [55]:
#converting to numpy array
x_err_conf_train = np.asarray(x_err_conf)
y_err_conf_train = np.asarray(y_err_conf)
x_std_conf_train = np.asarray(x_std_conf)
y_std_conf_train = np.asarray(y_std_conf)

#saving numpy array 
np.save('x_err_conf_multi_train', x_err_conf_train)
np.save('y_err_conf_multi_train', y_err_conf_train)
np.save('x_std_conf_multi_train', x_std_conf_train)
np.save('y_std_conf_multi_train', y_std_conf_train)


In [57]:
print(x_err_conf_train)

[0.00057774 0.00115376 0.00043045 0.00124472 0.00064328 0.00270663
 0.00059773 0.00082208 0.00389709]


In [73]:
print(x_err_conf_train)

[0.00049074 0.00024972 0.00849996 0.00374048 0.0011209  0.00028189
 0.00283479 0.00103393 0.00150859]


In [58]:
print(y_err_conf_train)

[0.00896326 0.0043567  0.01191031 0.02272385 0.01264716 0.04593218
 0.00867561 0.014009   0.01737641]


In [74]:
print(y_err_conf_train)

[0.0025165  0.0032034  0.03080785 0.04365365 0.03268663 0.0032748
 0.08341615 0.06028629 0.09108653]


In [59]:
import math
for i in range(9):
    print('nb hidden layers '+str(num_layer[i//3])+' and nb neurons '+str(hidden_dim[i%3])+':')
    print('mean squared error in x-axis:' + str(round(x_err_conf_train[i],5)))
    print('mean squared error in y-axis:' + str(round(y_err_conf_train[i],5)))
    print('std of mean squared error in x-axis:' + str(round(x_std_conf_train[i],5)))
    print('std of mean squared error in y-axis:' + str(round(y_std_conf_train[i],5)))
    print('.........')

nb hidden layers 3 and nb neurons 16:
mean squared error in x-axis:0.00058
mean squared error in y-axis:0.00896
std of mean squared error in x-axis:0.00018
std of mean squared error in y-axis:0.01229
.........
nb hidden layers 3 and nb neurons 32:
mean squared error in x-axis:0.00115
mean squared error in y-axis:0.00436
std of mean squared error in x-axis:0.00044
std of mean squared error in y-axis:0.00143
.........
nb hidden layers 3 and nb neurons 64:
mean squared error in x-axis:0.00043
mean squared error in y-axis:0.01191
std of mean squared error in x-axis:0.00016
std of mean squared error in y-axis:0.01364
.........
nb hidden layers 5 and nb neurons 16:
mean squared error in x-axis:0.00124
mean squared error in y-axis:0.02272
std of mean squared error in x-axis:0.00046
std of mean squared error in y-axis:0.01517
.........
nb hidden layers 5 and nb neurons 32:
mean squared error in x-axis:0.00064
mean squared error in y-axis:0.01265
std of mean squared error in x-axis:0.00023
std 

Compute and print the mean error for each hyperparameters combination :

In [60]:
err_traj = 0.5*(x_err_conf_train+y_err_conf_train)
print(err_traj)

[0.0047705  0.00275523 0.00617038 0.01198428 0.00664522 0.02431941
 0.00463667 0.00741554 0.01063675]


Choose the combination with the lowest error : 

In [61]:
best_conf = np.argmin(err_traj)
print('the lowest error is obtain with '+str(num_layer[best_conf//3])+ ' hidden layer'+' and '+str(hidden_dim[best_conf%3])+' neurons'+' with a mean squared error of '+str(err_traj[best_conf]))

the lowest error is obtain with 3 hidden layer and 32 neurons with a mean squared error of 0.0027552268552429454


In [62]:
std_err_traj = 0.5*(x_std_conf_train+y_std_conf_train)
print(std_err_traj)

[0.00623241 0.00093159 0.00690351 0.00781494 0.00981656 0.01802698
 0.00259372 0.00487926 0.00523886]
